In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
df = pd.read_csv('winemag-data_first150k.csv')
df[df['variety']=='Rh\xc3\xb4ne-style Red Blend']['variety'] = 'Rhone-style Red Blend'
df = df[['variety', 'description']]
df.head()

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,variety,description
0,Cabernet Sauvignon,This tremendous 100% varietal wine hails from ...
1,Tinta de Toro,"Ripe aromas of fig, blackberry and cassis are ..."
2,Sauvignon Blanc,Mac Watson honors the memory of a wine once ma...
3,Pinot Noir,"This spent 20 months in 30% new French oak, an..."
4,Provence red blend,"This is the top wine from La Bégude, named aft..."


In [3]:
def remove_weird_char(string):
    words = string.lower().split()
    refined = []
    for i in words:
        refined.append(re.sub("[^\w-]+", "", i))
        sentence = ' '.join([word for word in refined])
    return(sentence)

In [4]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")
cachedStopWords.append('wine')
def rmStopWords(text):
        text = ' '.join([word for word in text.split() if word not in cachedStopWords])
        return(text)

In [5]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    stems = [stemmer.stem(t) for t in tokens]
    return stems

In [6]:
df['description'] = df['description'].apply(remove_weird_char).apply(rmStopWords)

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english',
                                   max_df=0.8, min_df=0.2,
                                   tokenizer=tokenize_and_stem,ngram_range=(1,3))
tfidf_matrix = tfidf_vectorizer.fit_transform(df['description']) #fit the vectorizer to synopses
print(tfidf_matrix.shape)

(150930, 7)


In [53]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [ ]:
tfidf_matrix

In [56]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(tfidf_matrix[1509])

array([[ 1.]])

In [54]:
terms

[u'acid', u'aroma', u'cherri', u'finish', u'flavor', u'fruit', u'tannin']